<a href="https://colab.research.google.com/github/helidenial/drt/blob/main/torre.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt install python3-pip >/etc/null 2>&1 ; echo "|"
!python -m pip install --upgrade pip setuptools wheel >/etc/null 2>&1 ; echo "||"
!python -m pip install lbry-libtorrent >/etc/null 2>&1 ; echo "|||"
!apt install python3-libtorrent >/etc/null 2>&1 ; echo "||||"

import time
from IPython.display import display
import ipywidgets as widgets
import libtorrent as lt
import requests
from google.colab import drive
import requests as re
from bs4 import BeautifulSoup as Soup

drive.mount("/content/drive")

ses = lt.session()
ses.listen_on(6881, 6891)
downloads = []

def sendMessage(bot_message):
    token = '1176657042:AAEGwetBc8y3jS6Oui4nIBd2w5NhGU6lmgk'
    chatID = '701156270'
    send_text = 'https://api.telegram.org/bot' + token + '/sendMessage?chat_id=' + chatID + '&parse_mode=Markdown&text=' + bot_message
    res = requests.get(send_text)
    return res

def getYifyFor(movie_name):
  base_link = "https://yts.mx/movies/"
  movie = movie_name.replace(' ','-').lower()
  res = re.get(base_link+movie)

  soup = Soup(res.content)

  link_a = soup.find('div',{"class":"bottom-info"}).findAll('a')
  for i,a in enumerate(link_a):
    try:
      print(f'[{i}]',a['title'],a['href'])
    except:
      pass

  inp = input("Enter which link to download")
  
  return link_a[int(inp)]['href']

def downloadFromYify(file_link):
  !wget $file_link
  file_name = file_link.split('/')[-1]
  params = {
    "save_path": "/content/drive/Shareddrives/tdHackgence/Torrent",
    "ti": lt.torrent_info(file_name),
  }
  downloads.append(ses.add_torrent(params))
 
location=""

def show_bars():
  name =''
  state_str = [
      "queued",
      "checking",
      "downloading metadata",
      "downloading",
      "finished",
      "seeding",
      "allocating",
      "checking fastresume",
  ]
  
  layout = widgets.Layout(width="auto")
  style = {"description_width": "initial"}
  download_bars = [
      widgets.FloatSlider(
          step=0.01, disabled=True, layout=layout, style=style
      )
      for _ in downloads
  ]
  display(*download_bars)
  
  while downloads:
      next_shift = 0
      for index, download in enumerate(downloads[:]):
          bar = download_bars[index + next_shift]
          if not download.is_seed():
              s = download.status()
  
              bar.description = " ".join(
                  [
                      download.name(),
                      str(s.download_rate / 1000),
                      "kB/s",
                      state_str[s.state],
                  ]
              )
              bar.value = s.progress * 100
          else:
              next_shift -= 1
              ses.remove_torrent(download)
              downloads.remove(download)
              bar.close() # Seems to be not working in Colab (see https://github.com/googlecolab/colabtools/issues/726#issue-486731758)
              download_bars.remove(bar)
              name = download.name()
              print(download.name(), "complete")
      time.sleep(1)


  sendMessage('Uploaded \n\n'+name+'\n\nTo ' + location )
  print('Done uploading '+name+' To your google drive')


import os
home_path = "/content/drive/Shareddrives/tdHackgence/Torrent/Subtitles"

def getSubsFor(sub_link):  
  temp_path = os.getcwd()
  os.chdir(home_path)
  res = re.get(sub_link)
  sub_link
  sou = Soup(res.content)
  movie="train-to-busan"
  res = sou.find("span",{"class":"flag flag-gb"})
  a_href = res.parent.parent.find('a')

  final_link="https://yifysubtitles.org"+a_href['href'].replace('subtitles','subtitle')+".zip"
  file_name = final_link.split('/')[-1]
  ! wget $final_link
  ! unzip $file_name
  os.chdir(temp_path)

def magnetDownloads(magnet_link):
  params = {"save_path": "/content/drive/Shareddrives/tdHackgence/Torrent"}
  downloads.append(lt.add_magnet_uri(ses, magnet_link, params))
  print(f"Appended {magnet_link} to downloads")


|
||
|||
||||


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.activity.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fexperimentsandconfigs%20https%3a%2f%2fwww.googleapis.com%2fauth%2fphotos.native&response_type=code

Enter your authorization code:
z


ValueError: ignored

In [ ]:
 ''' 
  All the functions so far : 
  
   sendMessage(msg) -> send msg to telegram. 
   getYifyFor(movie_name) -> get_links 
   downloadFromYify(file_link) -> Download from yify 
   magnetDownloads(magnet_link) -> Downloads from the magnet link 
   show_bars() -> Shows download bars 
    
   '''

In [ ]:
params = {"save_path": "/content/drive/Shareddrives/tdHackgence/Torrent"}
 
while True:
    magnet_link =''
    magnet_link = input("Enter Magnet Link Or Type Exit: ")
    if magnet_link.lower() == "exit":
        break
    downloads.append(
        lt.add_magnet_uri(ses, magnet_link, params)
    )
 
location = "Shareddrives/tdHackgence/Torrent"

Enter Magnet Link Or Type Exit: exit


In [ ]:
magnet = 'magnet:?xt=urn:btih:9941695f098ca5655bedbd8cbbcdc45f3c579578&dn=The.Falcon.and.The.Winter.Soldier.S01E06.Episode.6.1080p.DSNP.WEB-DL.DDP5.1.H.264-EVO%5Beztv%5D&tr=udp://tracker.coppersurfer.tk:80&tr=udp://glotorrents.pw:6969/announce&tr=udp://tracker.leechers-paradise.org:6969&tr=udp://tracker.opentrackr.org:1337/announce&tr=udp://exodus.desync.com:6969'
magnetDownloads(magnet)

Appended magnet:?xt=urn:btih:9941695f098ca5655bedbd8cbbcdc45f3c579578&dn=The.Falcon.and.The.Winter.Soldier.S01E06.Episode.6.1080p.DSNP.WEB-DL.DDP5.1.H.264-EVO%5Beztv%5D&tr=udp://tracker.coppersurfer.tk:80&tr=udp://glotorrents.pw:6969/announce&tr=udp://tracker.leechers-paradise.org:6969&tr=udp://tracker.opentrackr.org:1337/announce&tr=udp://exodus.desync.com:6969 to downloads


In [ ]:

'''

var id = setInterval(()=>{console.log("working");document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click();},3*60*1000);
var id2 = setInterval(()=>{console.log("x");document.querySelector("body > div.notebook-vertical > div.notebook-horizontal > div.layout.vertical.grow > colab-tab-layout-container > colab-tab-pane > colab-resizer > div.layout.vertical.grow > div > paper-tabs > paper-tab > paper-icon-button").click();},0.5*60*1000)

var btn = document.querySelector("#ok") 
if (btn != null ) document.querySelector("#ok").click()

'''

In [ ]:
params = {"save_path": "/content/drive/Shareddrives/tdHackgence/Torrent/AOT-Manga"}
 
while True:
    magnet_link =''
    magnet_link = input("Enter Magnet Link Or Type Exit: ")
    if magnet_link.lower() == "exit":
        break
    downloads.append(
        lt.add_magnet_uri(ses, magnet_link, params)
    )

location = "Shareddrives/tdHackgence/Torrent"

Enter Magnet Link Or Type Exit: exit


In [ ]:
getYifyFor(input("Enter the movie name with year of release:"))

Enter the movie name with year of release:Anand 1971
[0] Rotten Tomatoes Audience - Upright https://www.rottentomatoes.com/m/anand
[1] IMDb Rating https://www.imdb.com/title/tt0066763/
[3] Download Anand 720p.BluRay Torrent https://yts.mx/torrent/download/6B88C9F3B9640381B7C95329BEB42938B3489279
[4] Download Anand 1080p.BluRay Torrent https://yts.mx/torrent/download/712358FA136397C725740F2A61488AC827EC4A33
[5] Download subtitles for Anand 1080p https://yifysubtitles.org/movie-imdb/tt0066763
Enter which link to download4


'https://yts.mx/torrent/download/712358FA136397C725740F2A61488AC827EC4A33'

In [ ]:
show_bars()
downloads=[]

In [ ]:
!wget https://yts.mx/torrent/download/712358FA136397C725740F2A61488AC827EC4A33


--2021-05-12 08:35:23--  https://yts.mx/torrent/download/712358FA136397C725740F2A61488AC827EC4A33
Resolving yts.mx (yts.mx)... 172.67.72.176, 104.26.6.75, 104.26.7.75
Connecting to yts.mx (yts.mx)|172.67.72.176|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/octet-stream]
Saving to: ‘712358FA136397C725740F2A61488AC827EC4A33’

712358FA136397C7257     [  <=>               ]  76.73K   223KB/s    in 0.3s    

2021-05-12 08:35:24 (223 KB/s) - ‘712358FA136397C725740F2A61488AC827EC4A33’ saved [78570]



In [ ]:
getSubsFor(getYifyFor(input("Enter movie name with year of release ")))

In [5]:
fun = lambda x : x>2
 

In [8]:
arr = [1, 2, 3, 4, 5]

In [9]:
filter(fun,arr)

In [10]:
k = _


In [11]:
list(k)



[3, 4, 5]

In [106]:
import requests as re
from bs4 import BeautifulSoup as Soup
def yts(name):
  res = re.get('https://yts.mx/browse-movies/{}'.format(name))
  soup = Soup(res.content)
  links = soup.findAll('a')
  link = filter(lambda x : x['href'].startswith('https://yts.mx/movies/'), links)
  return list(map(lambda x:x['href'], link))

def ytsTorrent(link):
  di = {}
  res = re.get(link)
  soup = Soup(res.content)
  links = soup.findAll('a')
  link = filter(lambda x: x['href'].startswith('https://yts.mx/torrent/download/'), links)
  ll = list(map(lambda x: (x['href'], x['title']), link))
  for i in ll : 
    di[i[0]] = i[1]
  
  return di

In [107]:
ytsTorrent(yts('Avengers: EndGame')[0])

{'https://yts.mx/torrent/download/134F7AF967DBDB585A934A24C9F2A2EEE72E2A93': 'Download Avengers: Endgame 3D.BluRay Torrent',
 'https://yts.mx/torrent/download/223F7484D326AD8EFD3CF1E548DED524833CB77E': 'Download Avengers: Endgame 1080p.BluRay Torrent',
 'https://yts.mx/torrent/download/414A6F933C48FC7543A9CDB42C854B5457C5BCC7': 'Download Avengers: Endgame 1080p.WEB Torrent',
 'https://yts.mx/torrent/download/5A4140BD59D66BCAC57CF05AF4A8FAB4EBCAE1C1': 'Download Avengers: Endgame 720p.BluRay Torrent',
 'https://yts.mx/torrent/download/709FBD48374D2AE13C11B07A26B5E7DDC727D720': 'Download Avengers: Endgame 2160p.BluRay Torrent',
 'https://yts.mx/torrent/download/9D62160EEE330397A7A7BABA989A269C58CCFD8E': 'Download Avengers: Endgame 720p.WEB Torrent'}

{' ': None,
 '.': None,
 '0': None,
 '1': None,
 '8': None,
 ':': None,
 'A': None,
 'B': None,
 'D': None,
 'E': None,
 'R': None,
 'T': None,
 'a': None,
 'd': None,
 'e': None,
 'g': None,
 'l': None,
 'm': None,
 'n': None,
 'o': None,
 'p': None,
 'r': None,
 's': None,
 't': None,
 'u': None,
 'v': None,
 'w': None,
 'y': None}

NameError: ignored